In [1]:
PROJECT_ID = "demogct2022"
REGION = "us-central1"
ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)
PARENT = "projects/{}/locations/{}".format(PROJECT_ID, REGION)

In [2]:
from google.cloud import aiplatform_v1beta1
from google.protobuf import struct_pb2
import time


index_client = aiplatform_v1beta1.IndexServiceClient(
    client_options=dict(api_endpoint=ENDPOINT)
)


In [4]:
!gsutil ls gs://vertex-gen-ai/mjembedding/text/palm/

gs://vertex-gen-ai/mjembedding/text/palm/114k_PALM_Embedding.csv


In [8]:
DIMENSIONS = 768
DISPLAY_NAME = "MJ_TEXT_MultiModal_768"
BUCKET_ROOT= "gs://vertex-gen-ai/mjembedding/text/palm/"

In [9]:
treeAhConfig = struct_pb2.Struct(
    fields={
        "leafNodeEmbeddingCount": struct_pb2.Value(number_value=2000),
        "leafNodesToSearchPercent": struct_pb2.Value(number_value=20),
    }
)


algorithmConfig = struct_pb2.Struct(
    fields={"treeAhConfig": struct_pb2.Value(struct_value=treeAhConfig)}
)


config = struct_pb2.Struct(
    fields={
        "dimensions": struct_pb2.Value(number_value=DIMENSIONS),
        "approximateNeighborsCount": struct_pb2.Value(number_value=500),
        "distanceMeasureType": struct_pb2.Value(string_value="DOT_PRODUCT_DISTANCE"),
        "algorithmConfig": struct_pb2.Value(struct_value=algorithmConfig),
    }
)


metadata = struct_pb2.Struct(
    fields={
        "config": struct_pb2.Value(struct_value=config),
        "contentsDeltaUri": struct_pb2.Value(string_value=BUCKET_ROOT),
    }
)

ann_index = {
    "display_name": DISPLAY_NAME,
    "description": "Mid Journey 114K Image Embedding",
    "metadata": struct_pb2.Value(struct_value=metadata),
}


In [10]:
ann_index = index_client.create_index(parent=PARENT, index=ann_index)

In [11]:
ann_index.result()

name: "projects/353713988661/locations/us-central1/indexes/7487471875015049216"

In [12]:
INDEX_RESOURCE_NAME = ann_index.result().name
INDEX_RESOURCE_NAME


'projects/353713988661/locations/us-central1/indexes/7487471875015049216'

In [13]:
%%writefile text_index_endpoint.json
{
  "display_name": "MJ-palm-text-index-public-endpoint", 
    "publicEndpointEnabled":"true"
 
}


Overwriting text_index_endpoint.json


In [14]:
!curl -X POST     -H "Authorization: Bearer $(gcloud auth print-access-token)"     -H "Content-Type: application/json; charset=utf-8"     -d @text_index_endpoint.json     "https://us-central1-aiplatform.googleapis.com/v1/projects/demogct2022/locations/us-central1/indexEndpoints"


{
  "name": "projects/353713988661/locations/us-central1/indexEndpoints/7618076264208793600/operations/65792782941290496",
  "metadata": {
    "@type": "type.googleapis.com/google.cloud.aiplatform.v1.CreateIndexEndpointOperationMetadata",
    "genericMetadata": {
      "createTime": "2023-06-11T08:35:39.568726Z",
      "updateTime": "2023-06-11T08:35:39.568726Z"
    }
  }
}


In [19]:
%%writefile text_index_endpoint_deploy.json
{
  "deployedIndex": {
    "id": "mj_palm_text_index_public_endpoint_v2",
    "index": "projects/353713988661/locations/us-central1/indexes/7487471875015049216",
    "displayName": "public endpoint for mj palm index service"
  }
}


Overwriting text_index_endpoint_deploy.json


In [20]:
!curl -X POST     -H "Authorization: Bearer $(gcloud auth print-access-token)"     -H "Content-Type: application/json; charset=utf-8"     -d @text_index_endpoint_deploy.json     "https://us-central1-aiplatform.googleapis.com/v1/projects/demogct2022/locations/us-central1/indexEndpoints/7618076264208793600:deployIndex"

{
  "name": "projects/353713988661/locations/us-central1/indexEndpoints/7618076264208793600/operations/6676232610991046656",
  "metadata": {
    "@type": "type.googleapis.com/google.cloud.aiplatform.v1.DeployIndexOperationMetadata",
    "genericMetadata": {
      "createTime": "2023-06-11T08:39:21.257948Z",
      "updateTime": "2023-06-11T08:39:21.257948Z"
    },
    "deployedIndexId": "mj_palm_text_index_public_endpoint_v2"
  }
}


In [14]:
!curl -H "Content-Type: application/json" -H "Authorization: Bearer `gcloud auth print-access-token`" "https://us-central1-aiplatform.googleapis.com/v1/projects/demogct2022/locations/us-central1/indexEndpoints/4421646428682584064"

{
  "name": "projects/353713988661/locations/us-central1/indexEndpoints/4421646428682584064",
  "displayName": "MJ-text-index-public-endpoint",
  "deployedIndexes": [
    {
      "id": "mj_text_index_public_endpoint",
      "index": "projects/353713988661/locations/us-central1/indexes/1794921946018742272",
      "displayName": "public endpoint for imdb index service",
      "createTime": "2023-06-10T04:35:42.615422Z",
      "indexSyncTime": "2023-06-10T04:50:50.956125Z",
      "automaticResources": {
        "minReplicaCount": 2,
        "maxReplicaCount": 2
      },
      "deploymentGroup": "default"
    }
  ],
  "etag": "AMEw9yMHvP-5bUGRsZzgH74I2retykj_UCx3QiNWZ877vEch2hnlVcpfcTc1wjJmBHrX",
  "createTime": "2023-06-10T04:30:30.567719Z",
  "updateTime": "2023-06-10T04:30:31.271865Z",
  "publicEndpointDomainName": "547915122.us-central1-353713988661.vdb.vertexai.goog"
}


In [16]:
from google.oauth2 import service_account
from google.cloud import aiplatform_v1beta1
 
def findneighbor_sample(val):
  # The AI Platform services require regional API endpoints.
  scopes = ["https://www.googleapis.com/auth/cloud-platform"]
  
  # create a service account with `Vertex AI User` role granted in IAM page.
  # download the service account key https://developers.google.com/identity/protocols/oauth2/service-account#authorizingrequests
  sa_file_path = "/home/jupyter/demogct2022-ca8b4e443d11.json"
 
  credentials = service_account.Credentials.from_service_account_file(
      sa_file_path, scopes=scopes
  )
  client_options = {
      "api_endpoint": "547915122.us-central1-353713988661.vdb.vertexai.goog"
  }
 
  vertex_ai_client = aiplatform_v1beta1.MatchServiceClient(
      credentials=credentials,
      client_options=client_options,
  )
 
  request = aiplatform_v1beta1.FindNeighborsRequest(
      index_endpoint="projects/353713988661/locations/us-central1/indexEndpoints/4421646428682584064",
      deployed_index_id="mj_text_index_public_endpoint",
  )
  dp1 = aiplatform_v1beta1.IndexDatapoint(
      datapoint_id="0",
      feature_vector=val,
  )
  query = aiplatform_v1beta1.FindNeighborsRequest.Query(
      datapoint=dp1,
  )
  request.queries.append(query)
 
  response = vertex_ai_client.find_neighbors(request)
  return response